# RSGISLib
<img src="https://rsgislib.org/_static/rsgislib-logo.png" width="400" >

Trata-se de uma biblioteca que reúne diversos métodos de e ferramentas para o processamento de datasets sobre sensoriamento remoto e SIG (Sistemas de informação geográfica).

A biblioteca contém uma série de algoritmos para facilitar as pesquisas na área de sensoriamento remoto, que inclui as seguintes funções:
* Segmentação de imagem;
* Classificação de objetos;
* Registro Imagem a imagem;
* Filtragem de imagens;
* Estatísticas zonais
* Processamento de vetores;

A biblioteca é composta por diversos módulos que implementam métodos úteis para cada, portanto vamos tentar cobrir alguns dos principais módulos, pelo menos neste momento, do desenvolvimento do projeto de satélites.


## RSGISLib Image Calibration Module

Módulo composto por funções importantes para calibração de imagens de satélite, entre elas conversão de valores DN para radiancia e reflectância topo da atmosfera, bem como reflectância de superfície, utilizando os coeficientes 6S. 

## Image Band Math

In [1]:
import rsgislib
from rsgislib import imagecalc
from rsgislib import imageutils

In [8]:
# Calcula a potência para a imagem HH:

hhImg = './dataset/N22E089_15_sl_HH_F02DAR'
hhImgPow = './dataset/N22E089_15_sl_HH_F02DAR_pow.kea'
bandDefns = []
# Cria uma lista dos objetos bandas para passar à função bandMath como o parâmetro 'bands'.
bandDefns.append(rsgislib.imagecalc.BandDefn('HH', hhImg, 1))
# Expressão de potência HH em sintaxe muparser
mathExp = 'HH>0?10^(2*log10(HH) - 8.3):0.0'
rsgislib.imagecalc.bandMath(hhImgPow, mathExp, 'KEA', rsgislib.TYPE_32FLOAT, bandDefns)

In [9]:
# Calcula a potência para a imagem HV:

hvImg = './dataset/N22E089_15_sl_HV_F02DAR'
hvImgPow = './dataset/N22E089_15_sl_HV_F02DAR_pow.kea'
bandDefns = []
# Cria uma lista dos objetos bandas para passar à função bandMath como o parâmetro 'bands'.
bandDefns.append(imagecalc.BandDefn('HV', hvImg, 1))
# Expressão de potência HV em sintaxe muparser
mathExp = 'HV>0?10^(2*log10(HV) - 8.3):0.0'
imagecalc.bandMath(hvImgPow, mathExp, 'KEA', rsgislib.TYPE_32FLOAT, bandDefns)

In [10]:
# Calcula HH/HV:

hhhvImg = './dataset/N22E089_15_sl_HHHV_F02DAR_pow.kea'
bandDefns = []
# Pega as bandas HH calculadas anteriormente 
bandDefns.append(imagecalc.BandDefn('HH', hhImgPow, 1))
# Pega as bandas HH calculadas anteriormente 
bandDefns.append(imagecalc.BandDefn('HV', hvImgPow, 1))
# Expressão de HH/HV em sintaxe muparser
mathExp = 'HV>0?HH/HV:0.0'
imagecalc.bandMath(hhhvImg, mathExp, 'KEA', rsgislib.TYPE_32FLOAT, bandDefns)

In [11]:
# Uni as bandas em uma única imagem
imageList = [hhImgPow, hvImgPow, hhhvImg]
bandNamesList = ['HH','HV', 'HH/HV']
outputImageStack = './dataset/N22E089_15_sl_F02DAR_powstack.kea'
imageutils.stackImageBands(imageList, bandNamesList, outputImageStack, None, 0, 'KEA', rsgislib.TYPE_32FLOAT)

In [12]:
# Calculate overview pyramids and image statistics to make visualisation faster.
imageutils.popImageStats(outputImageStack, usenodataval=True, nodataval=0,calcpyramids=True)

## Batch Processing

O processo anterior permite utilizar a biblioteca e várias de suas funções, ali usamos apenas algumas mais simples, para realizar operações com bandas das imagens, além de definir suas próprias bandas de acordo com uma expressão matemática. Vamos imaginar agora, porém, que seja necessário realizar o procedimento para um grande número de imagens, será necessário adicionar alguns passos intermediários, entre eles o principal é criar uma função capaz de realizar os procedimentos anteriormente citados, para que seja possível chamar a função para cada imagem a ser trabalhada.

In [2]:
import os
import os.path
import subprocess
import shutil
import glob
import multiprocessing

In [3]:
def createPALSARStack(inputTAR, outputStackImg, tmpDir):
    
    # Make sure that the inputs use absolute paths as using a working directory
    inputTAR = os.path.abspath(inputTAR)
    outputStackImg = os.path.abspath(outputStackImg)
    tmpDir = os.path.abspath(tmpDir)
    
    # Check file input file and tmp directory are present.
    if not os.path.exists(tmpDir):
        raise rsgislib.RSGISPyException('Tmp directory is not present.')
        
    if not os.path.exists(inputTAR):
        raise rsgislib.RSGISPyException('Input tar file is not present.')
    # Get the rsgis utilities object
    rsgisUtils = rsgislib.RSGISPyUtils()
    
    # Get a unique id for processing
    uidStr = rsgisUtils.uidGenerator()
    
    # Create a working directory
    workDIR = os.path.join(tmpDir, uidStr)
    if not os.path.exists(workDIR):
        os.makedirs(workDIR)
    # Save current working path
    cPWD = os.getcwd()
    # Move into that working directory.
    os.chdir(workDIR)
    
    # Extract tar.gz file - using the terminal commands.
    cmd = 'tar -zxf ' + inputTAR
    print(cmd)
    try:
        subprocess.call(cmd, shell=True)
    except OSError as e:
       raise rsgislib.RSGISPyException('Could not execute command: ' + cmd)
    
    # Find the HH and HV images.
    hhImg = ''
    hvImg = ''
    
    hhFiles = glob.glob(os.path.join(workDIR, '*_sl_HH_F02DAR'))
    hvFiles = glob.glob(os.path.join(workDIR, '*_sl_HV_F02DAR'))
    
    if len(hhFiles) == 1:
        hhImg = hhFiles[0]
    else:
        raise rsgislib.RSGISPyException('Could not find HH file')
    if len(hvFiles) == 1:
        hvImg = hvFiles[0]
    else:
        raise rsgislib.RSGISPyException('Could not find HV file')
    
    print("HH File: ", hhImg)
    print("HV File: ", hvImg)
    
    # Calculate Power for HH image
    hhImgPow = os.path.join(workDIR, uidStr+'_HH_Pow.kea')
    bandDefns = []
    bandDefns.append(imagecalc.BandDefn('HH', hhImg, 1))
    mathExp = 'HH>0?10^(2*log10(HH) - 8.3):0.0'
    imagecalc.bandMath(hhImgPow, mathExp, 'KEA', rsgislib.TYPE_32FLOAT, bandDefns)
    
    # Calculate Power for HV image
    hvImgPow = os.path.join(workDIR, uidStr+'_HV_Pow.kea')
    bandDefns = []
    bandDefns.append(imagecalc.BandDefn('HV', hvImg, 1))
    mathExp = 'HV>0?10^(2*log10(HV) - 8.3):0.0'
    imagecalc.bandMath(hvImgPow, mathExp, 'KEA', rsgislib.TYPE_32FLOAT, bandDefns)

    # Calculate HH / HV
    hhhvImg = os.path.join(workDIR, uidStr+'_HHHV_Pow.kea')
    bandDefns = []
    bandDefns.append(imagecalc.BandDefn('HH', hhImgPow, 1))
    bandDefns.append(imagecalc.BandDefn('HV', hvImgPow, 1))
    mathExp = 'HV>0?HH/HV:0.0'
    imagecalc.bandMath(hhhvImg, mathExp, 'KEA', rsgislib.TYPE_32FLOAT, bandDefns)

    # Stack images into a single band
    imageList = [hhImgPow, hvImgPow, hhhvImg]
    bandNamesList = ['HH','HV', 'HH/HV']
    imageutils.stackImageBands(imageList, bandNamesList, outputStackImg, None, 0, 
                               'KEA', rsgislib.TYPE_32FLOAT)
    
    # Calculate overview pyramids and image statistics to make visualisation faster.
    imageutils.popImageStats(outputStackImg, usenodataval=True, nodataval=0, 
                         calcpyramids=True)
                  
    # Clean up by deleting the working directory
    os.chdir(cPWD) # Move back to starting directory before delete
    shutil.rmtree(workDIR)

In [4]:
createPALSARStack('N22E088_15_MOS_F02DAR.tar.gz', 'N22E088_15_MOS_F02DAR_Stack.kea', './tmp')
createPALSARStack('N22E089_15_MOS_F02DAR.tar.gz', 'N22E089_15_MOS_F02DAR_Stack.kea', './tmp')
createPALSARStack('N23E088_15_MOS_F02DAR.tar.gz', 'N23E088_15_MOS_F02DAR_Stack.kea', './tmp')
createPALSARStack('N23E089_15_MOS_F02DAR.tar.gz', 'N23E089_15_MOS_F02DAR_Stack.kea', './tmp')

tar -zxf /home/fill/Documentos/Turing/VC/Satélites/N22E088_15_MOS_F02DAR.tar.gz
HH File:  /home/fill/Documentos/Turing/VC/Satélites/tmp/3b1ee5/N22E088_15_sl_HH_F02DAR
HV File:  /home/fill/Documentos/Turing/VC/Satélites/tmp/3b1ee5/N22E088_15_sl_HV_F02DAR
tar -zxf /home/fill/Documentos/Turing/VC/Satélites/N22E089_15_MOS_F02DAR.tar.gz
HH File:  /home/fill/Documentos/Turing/VC/Satélites/tmp/d76ba6/N22E089_15_sl_HH_F02DAR
HV File:  /home/fill/Documentos/Turing/VC/Satélites/tmp/d76ba6/N22E089_15_sl_HV_F02DAR
tar -zxf /home/fill/Documentos/Turing/VC/Satélites/N23E088_15_MOS_F02DAR.tar.gz
HH File:  /home/fill/Documentos/Turing/VC/Satélites/tmp/13534b/N23E088_15_sl_HH_F02DAR
HV File:  /home/fill/Documentos/Turing/VC/Satélites/tmp/13534b/N23E088_15_sl_HV_F02DAR
tar -zxf /home/fill/Documentos/Turing/VC/Satélites/N23E089_15_MOS_F02DAR.tar.gz
HH File:  /home/fill/Documentos/Turing/VC/Satélites/tmp/cd6528/N23E089_15_sl_HH_F02DAR
HV File:  /home/fill/Documentos/Turing/VC/Satélites/tmp/cd6528/N23E089_

## Data Pre-processing

A transformação de uma imagem com valores SAR ou DN em potência HH é um tipo de pre-processamento a ser realizado antes de conseguir fazer algo, de fato, útil com a imagem. Neste sentido a biblioteca proporciona diversos métodos diferentes, vamos explorar alguns deles:

### Moisaico de imagens

Mosaico de imagens é uma ferramenta disponível para unir as diferentes imagens criadas como um mosaico de bandas.

In [6]:
# Procurando por todos arquivos com a extensão 'kea'
inputList = glob.glob('*.kea')
outImage = './Sundarbans_15_MOS_F02DAR.kea'

backgroundVal = 0.0
skipVal = 0.0
skipBand = 1
overlapBehaviour = 0

imageutils.createImageMosaic(inputList, outImage, backgroundVal,
                             skipVal, skipBand, overlapBehaviour, 
                             'KEA', rsgislib.TYPE_32FLOAT)

imageutils.popImageStats(outImage, usenodataval=True, nodataval=0, calcpyramids=True)

### Re-projecting/Re-sampling

Num problema de classificação, imagine que uma imagem foi obtida pelo satélite PALSAR-2, enquanto outra imagem no dataset foi obtida pelo Landsat-8. PALSAR fornece imagens em lat/long WGS84, enquanto Landsat-8 é fornecida em WSG84 UTM Zone 45. Assim é necessário reprojetar e reamostrar as imagens para que ambas estejam numa mesma 'escala'. Essa função abaixo pode ser utilizada sempre que for necessário unificar as imagens em termos de pixel grid, tamanho de pixel, extensão e projeção. 

In [7]:
inRefImg = './Datasets/Sundarbans_201511_Landsat.kea'
inProcessImg = 'Sundarbans_15_MOS_F02DAR.kea'
outImg = 'Sundarbans_15_MOS_F02DAR_utm45n.kea'

imageutils.resampleImage2Match(inRefImg, inProcessImg, outImg, 
                               'KEA', 'cubic', rsgislib.TYPE_32FLOAT)

imageutils.popImageStats(outImg, usenodataval=True, 
                         nodataval=0, calcpyramids=True)

101it [00:27,  4.56it/s]                        

### Criando uma máscara de dados válidos

É comum em imagens de satélite que alguns dados sejam não significativos, não válidos, neste caso por exemplo quando todas as 3 bandas tem o valor 0 no pixel. Para tal temos essa função capaz de criar uma máscara de dados válidos apenas, criada da seguinte forma:

In [8]:
from rsgislib import rastergis

landsatImg = './Datasets/Sundarbans_201511_Landsat.kea'
palsarImg = 'Sundarbans_15_MOS_F02DAR_utm45n.kea'
validMask = 'Sundarbans_validmsk.kea'

imageutils.genValidMask(inimages=[landsatImg,palsarImg], 
                        outimage=validMask, gdalformat='KEA', 
                        nodata=0.0)

rastergis.populateStats(clumps=validMask, addclrtab=True, 
                        calcpyramids=True, ignorezero=True) 


101it [15:35,  9.27s/it]


### Raster para vetor

Vamos usar a função que transforma um raster em um vetor para criar um shapefile de polígono para a área válida da nossa imagem:

In [9]:
from rsgislib import vectorutils

inputImg = 'Sundarbans_validmsk.kea'
outShp = 'Sundarbans_validmsk_shp.shp'              
vectorutils.polygoniseRaster(inputImg, outShp, imgBandNo=1, 
                             maskImg=inputImg, imgMaskBandNo=1)

  5%|▌         | 5/100 [00:00<00:02, 43.29it/s]

Polygonising...


100%|██████████| 100/100 [00:00<00:00, 85.12it/s]

Completed


101it [00:12, 85.12it/s]                         

### Vetor para Raster

Na RSGISLib é mais comum trabalhar com rasters que com vetores, portanto 'rasterizar' um vetor na mesma pixel grid que a imagem que pretendemos usar como a camada de vetor é uma operação comum de ser executada. No exemplo vamos utilizar o vetor extraído acima para rasterizar mas na grid da Landsat.

In [3]:
inputVec = 'Sundarbans_validmsk_shp.shp'
inputImage = './Datasets/Sundarbans_201511_Landsat.kea'
outImage = 'Sundarbans_ValidMask_Landsat_tmp.kea'
vectorutils.rasterise2Image(inputVec, inputImage, outImage, 
                            gdalformat='KEA', burnVal=1)

NameError: name 'vectorutils' is not defined

### Image spatian subsetting 

É comum querer selecionar uma parte da imagem que seja de interesse da aplicação, a seguinte configuração pode fazer isso de forma simplificada:

In [5]:
inputvector = 'Sundarbans_validmsk_shp.shp'

inputimage = 'Sundarbans_15_MOS_F02DAR_utm45n.kea'
outputimage = 'Sundarbans_15_MOS_F02DAR_utm45n_sub.kea'
imageutils.subset(inputimage, inputvector, outputimage, 
                  'KEA', rsgislib.TYPE_32FLOAT)

imageutils.popImageStats(outputimage, usenodataval=True, 
                         nodataval=0, calcpyramids=True)
                        
inputimage = './Datasets/Sundarbans_201511_Landsat.kea'
outputimage = 'Sundarbans_201511_Landsat_sub.kea'
imageutils.subset(inputimage, inputvector, outputimage, 
                  'KEA', rsgislib.TYPE_16UINT)

imageutils.popImageStats(outputimage, usenodataval=True, 
                         nodataval=0, calcpyramids=True)

###  Image Masking

A operação de aplicar uma máscara sobre uma imagem é feita quando queremos eliminar uma parte da imagem que não apresenta contribuição significativa para nossa análise, e, no caso de análise automatizadas, pode até atrapalhar. RSGISLib apresenta um comando direto para aplicar uma máscara sobre a imagem:

In [6]:
imagemask = 'Sundarbans_validmsk.kea'

inputimage = 'Sundarbans_15_MOS_F02DAR_utm45n_sub.kea'
outputimage = 'Sundarbans_15_MOS_F02DAR_utm45n_sub_msk.kea'

imageutils.maskImage(inputimage, imagemask, outputimage, 
                     'KEA', rsgislib.TYPE_32FLOAT, 0.0, 0.0)

imageutils.popImageStats(outputimage, usenodataval=True, 
                         nodataval=0, calcpyramids=True)
                         
inputimage = 'Sundarbans_201511_Landsat_sub.kea'
outputimage = 'Sundarbans_201511_Landsat_sub_msk.kea'

imageutils.maskImage(inputimage, imagemask, outputimage, 
                     'KEA', rsgislib.TYPE_16UINT, 0.0, 0.0)

imageutils.popImageStats(outputimage, usenodataval=True, 
                         nodataval=0, calcpyramids=True)

###  Image Band Subsetting

A biblioteca também é capaz de realizar a seleção de bandas na imagem, por exemplo, ao converter para dB o valor dos pixel, vamos apenas utilizar as bandas HH e HV:

In [7]:
inputimage = 'Sundarbans_15_MOS_F02DAR_utm45n_sub_msk.kea'
outputimage = 'Sundarbans_15_MOS_F02DAR_utm45n_sub_msk_HHHV.kea'
imageutils.selectImageBands(inputimage, outputimage, 'KEA', 
                            rsgislib.TYPE_32FLOAT, [1,2])

imageutils.popImageStats(outputimage, usenodataval=True, 
                         nodataval=0, calcpyramids=True)

###  Apply Lee Filter

Em muitas aplicações é bastante útil filtrar o SAR, a biblioteca possui diversos filtros disponíveis, entre eles:
* RSGISAbstractFilter
* applyMedianFilter
* applyMeanFilter
* applyGaussianSmoothFilter
* applySobelFilter
* applySobelXFilter
* applySobelYFilter
* applyPrewittFilter
* applyPrewittXFilter
* applyGaussian1stDerivFilter

E diversos outros, neste caso vamos aplicar o filtro de Lee:

In [9]:
from rsgislib import imagefilter

inputimage = 'Sundarbans_15_MOS_F02DAR_utm45n_sub_msk_HHHV.kea'
outputimage = 'Sundarbans_15_MOS_F02DAR_utm45n_sub_msk_HHHV_lee.kea'
imagefilter.applyLeeFilter(inputimage, outputimage, 5, 5, 
                           'KEA', rsgislib.TYPE_32FLOAT)

imageutils.popImageStats(outputimage, usenodataval=True, 
                         nodataval=0, calcpyramids=True)